How to use npose_util

First you need this terrible include

In [1]:

import sys
sys.path.append("/home/dzorine/software/npose/")
from npose_util import *

In [2]:
from importlib import reload
import npose_util
reload(npose_util)
from npose_util import *

Then you can open a pose like this

In [3]:
npose = npose_from_file("/home/bcov/sc/random/npose/ideal_helix_16.pdb")

nposes are actually just numpy arrays. Currently they store N, CA, CB, C, O

In [4]:
print(len(npose))
print(nsize(npose))
npose[:5]

90
18


array([[ 0.   ,  0.   ,  0.   ,  1.   ],
       [ 1.458,  0.   ,  0.   ,  1.   ],
       [ 1.988, -0.773, -1.199,  1.   ],
       [ 2.009,  1.42 ,  0.   ,  1.   ],
       [ 2.909,  1.747,  0.774,  1.   ]])

Or equivalently:

In [5]:
npose[:R]

array([[ 0.   ,  0.   ,  0.   ,  1.   ],
       [ 1.458,  0.   ,  0.   ,  1.   ],
       [ 1.988, -0.773, -1.199,  1.   ],
       [ 2.009,  1.42 ,  0.   ,  1.   ],
       [ 2.909,  1.747,  0.774,  1.   ]])

You should always use R instead of 5. Later nposes will have variable number of atoms. Remember that you can find the number of residues with nsize(npose). Also, nposes start numbering at 0 not 1.

If you want individual atoms, use the constants N, CA, CB, C, O

In [10]:
print(npose[3*R+N])
print(npose[3*R+CA])
print(npose[3*R+CB])
print(npose[3*R+C])
print(npose[3*R+O])

[1.70799994 3.14700007 3.39499998 1.        ]
[2.70099998 2.63800001 4.33400011 1.        ]
[2.82999992 1.12699997 4.19999981 1.        ]
[4.05299997 3.30599999 4.12400007 1.        ]
[4.6960001  3.7420001  5.07800007 1.        ]


Look at the coordinates though. Why is there an extra 1 at the end? 
That's for homogeneous transforms


================================================

There are a few more types of nposes that are important. tposes and itposes

tpose = transform pose

itpose = inverse tranform pose

In [7]:
tpose = tpose_from_npose(npose)
itpose = itpose_from_tpose(tpose)

Let's look at what a tpose is

In [12]:
tpose[3] #transform for residue 3 (rosetta resdidue 4)

array([[ 0.68089586,  0.68785483, -0.25146881,  2.70099998],
       [-0.34901916,  0.60661852,  0.71428257,  2.63800001],
       [ 0.64386836, -0.39858461,  0.65311855,  4.33400011],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

That right there is a homogeneous transform. They're pretty neat.

In this scenario, tpose[3] will move points from the origin to the reference frame of residue 3

In [13]:
#Example:
pt_at_origin = np.array([0, 0, 0, 1])
pt_at_res3 = tpose[3] @ pt_at_origin

print(pt_at_res3)     # the point we moved
print(npose[R*3+CA])  # CA of res 3

[2.70099998 2.63800001 4.33400011 1.        ]
[2.70099998 2.63800001 4.33400011 1.        ]


The inverse transforms do the opposite. They move that residue to the origin

In [14]:
CA_of_res3 = npose[R*3+CA]
CA_at_origin = itpose[3] @ CA_of_res3

print(CA_at_origin)

[0.0000000e+00 0.0000000e+00 4.4408921e-16 1.0000000e+00]


Now comes the cool part. You can move something to the origin, and then somewhere else:

In [15]:
CA_of_res3 = npose[R*3+CA]
CA_of_res3_at_res5 = tpose[5] @ itpose[3] @ CA_of_res3

print(CA_of_res3_at_res5)
print(npose[R*5+CA])

[4.70800018 7.59100008 3.20300007 1.        ]
[4.70800018 7.59100008 3.20300007 1.        ]


You can even move multiple atoms at once

In [16]:
res3 = npose[R*3:R*4]
res3_at_5 = xform_npose( tpose[5] @ itpose[3], res3 )

# Quick note about xform_npose. This is actually just a wrapper for numpy.
# But the numpy code is really ugly and this is way prettier

for atom in [N, CA, CB, C, O]:
    print(res3_at_5[atom], npose[R*5+atom])

[4.7690051  6.19788467 2.77596468 1.        ] [4.76900005 6.19799995 2.77600002 1.        ]
[4.70800018 7.59100008 3.20300007 1.        ] [4.70800018 7.59100008 3.20300007 1.        ]
[3.38504644 8.215787   2.78210474 1.        ] [3.38599992 8.2159996  2.78299999 1.        ]
[4.89785884 7.71590529 4.70851671 1.        ] [4.89799976 7.71500015 4.70900011 1.        ]
[5.6725928  8.54746325 5.17964291 1.        ] [5.67299986 8.54699993 5.17999983 1.        ]


================================================

Lets do a simple graft to show off the power of this.

We will graft a copy of npose to the end of npose to make a double-length pose

In [8]:
last_res = nsize(npose)-1

xform = tpose[last_res] @ itpose[0]
xformed_npose = xform_npose(xform, npose)

# This one has the two atoms overlapping so that it's easier to see
overlapping_npose = np.concatenate([npose, xformed_npose])
dump_npdb(overlapping_npose, "overlapping_npose.pdb")

# This one has the overlap removed
long_npose = np.concatenate([npose[:-1*R], xformed_npose])
dump_npdb(long_npose, "long_npose.pdb")

It's also easy to transform tposes and itposes

In [18]:
xformed_tpose = xform @ tpose

# for itposes you do it backwards with an inverse xform
xformed_itpose = itpose @ np.linalg.inv(xform)

============================================

Phi and psi

You can work with phi and psi angles for nposes too

In [19]:
phis = get_npose_phis(npose)
psis = get_npose_psis(npose)

print(phis)
print(psis)

[  0.         -56.99425898 -56.95914032 -56.97491673 -56.91845056
 -56.9156417  -56.99584666 -57.00662154 -57.00072312 -57.05231678
 -56.99501484 -57.04398276 -56.92164163 -57.02109971 -57.04409286
 -56.93604797 -57.06804164 -57.01863396]
[-46.99647375 -47.04813326 -47.03630896 -47.03626505 -47.08831681
 -47.04757664 -46.97279356 -47.06529221 -46.92476093 -47.04326584
 -46.99913089 -46.97019058 -47.04365668 -46.98071164 -47.05524206
 -47.00171315 -46.90340192   0.        ]


Setting phi and psi is a little weird, you use this function

In [20]:
resno = 4; phi=10
new_npose, new_tpose, new_phis = set_npose_phi(npose, tpose, phis, resno, phi)
print(new_phis)
print(get_npose_phis(new_npose))

[  0.         -56.99425898 -56.95914032 -56.97491673  10.
 -56.9156417  -56.99584666 -57.00662154 -57.00072312 -57.05231678
 -56.99501484 -57.04398276 -56.92164163 -57.02109971 -57.04409286
 -56.93604797 -57.06804164 -57.01863396]
[  0.         -56.99425898 -56.95914032 -56.97491673  10.
 -56.9156417  -56.99584666 -57.00662154 -57.00072312 -57.05231678
 -56.99501484 -57.04398276 -56.92164163 -57.02109971 -57.04409286
 -56.93604797 -57.06804164 -57.01863396]


What's interesting though. Is that if you only have CA for instance. You can still set phi and psi

In [21]:
CAs = extract_CA(npose)
new_CAs, new_tpose, new_phis = set_ca_phi(CAs, tpose, phis, resno, phi)

But you can't get the the phi and psi back out of CAs

==========================================

Clash checking!!!!

Perhaps one of the most important reason to use npose is for fast clash checking

Currently clash checking is currently optimized for making changes with lever arms.
- This inclues grafting
- Or changing a phi or psi

You must define a context and a test pose

You want the context to be the pose with fewer atoms. So if you are comparing a big pose and a little pose, 
this distinction is clear. But if you are comparing one pose with hundreds of others, the one pose counts
as the little pose because it has fewer total atoms.

For this example. We're going to change phi and psi of residue 8 and see how many clashes we get:
- The context will be the residues 0-7
- The test pose will be residues 8-15

In [22]:
# build the context

CAs = extract_CA(npose)
context = CAs[:8][:,:3] # What is the context. Also this should just be xyz
pt = context[-1][:3] # Where are the changes going to occur from?
max_dist = 100 # what's the maximum distance you might expect to see

context_by_dist, context_dist_limits = prepare_context_by_dist_and_limits(context, pt, max_dist)


In [23]:
# Next we build the test pose

test = CAs[8:][:,:3] # What is the test region. Also this should just be xyz
test_dists = np.linalg.norm( test - pt, axis=1) # We have to know the distance from the test pt to all test CAs
clash_dist = 4.0 # A reasonable clash distance for CA - CA
max_clash = 1000 # Setting this correctly makes it faster if you have a limit
tol = 2 # How accurately did you pick the test point? Smaller is faster but you might miss something

clashes = clash_check_points_context(test, test_dists, context_by_dist, 
                                     context_dist_limits, clash_dist, max_clash, tol)
print(clashes)

1


1 clash. This makes sense. The next CA is 3.8 A away

Now lets change phi and see what happens

In [24]:
phis = get_npose_phis(npose)
for phi in np.arange(0, 360, 20):
    new_CAs, new_tpose, new_phis = set_ca_phi(CAs, tpose, phis, 8, phi)
    test = new_CAs[8:][:,:3]
    test_dists = np.linalg.norm( test - pt, axis=1)

    clashes = clash_check_points_context(test, test_dists, context_by_dist, 
                                     context_dist_limits, clash_dist, max_clash, tol)
    
    print("Phi: %3i Clashes: %3i"%(phi, clashes))

Phi:   0 Clashes:   1
Phi:  20 Clashes:   1
Phi:  40 Clashes:   1
Phi:  60 Clashes:   1
Phi:  80 Clashes:   1
Phi: 100 Clashes:   1
Phi: 120 Clashes:   1
Phi: 140 Clashes:   3
Phi: 160 Clashes:   2
Phi: 180 Clashes:   2
Phi: 200 Clashes:   3
Phi: 220 Clashes:   4
Phi: 240 Clashes:   3
Phi: 260 Clashes:   1
Phi: 280 Clashes:   1
Phi: 300 Clashes:   1
Phi: 320 Clashes:   1
Phi: 340 Clashes:   1


Don't set phi to 220 apparently

========================================

Integrating with pyRMSD

In [25]:
# Stupid shenanigains to import pyRMSD. You'll probably have to restart the kernal and then run this first

import sys
sys.path.insert(0, "/home/bcov/miniconda3/lib/python3.5/site-packages")
import pyRMSD.RMSDCalculator as fast_rmsd_calc

 Ok, let's start randomly generating poses until we have 100 that are at least 7A rmsd from each other

In [83]:
import random

# Start our collection arrays with the starting pose
saved = [extract_CA(npose)[:,:3]]
saved_phis = [get_npose_phis(npose)]
saved_psis = [get_npose_psis(npose)]

while (len(saved) < 100):
    # Make a copy of all the important things
    this_pose = extract_CA(npose)
    this_phis = get_npose_phis(npose)
    this_psis = get_npose_psis(npose)
    this_tpose = tpose_from_npose(npose)
    
    min_rmsd = 0
    while (min_rmsd < 7):
        # Change phi and psi until we're really random
        res = random.randint(0, len(this_pose)-1)
        phi = random.randint(0, 360)
        psi = random.randint(0, 360)
        
        this_pose, this_tpose, this_phis = set_ca_phi(this_pose, this_tpose, this_phis, res, phi)
        this_pose, this_tpose, this_psis = set_ca_psi(this_pose, this_tpose, this_psis, res, psi)
        
        calc = fast_rmsd_calc.RMSDCalculator("QCP_SERIAL_CALCULATOR", np.array([this_pose[:,:3]] + saved))
        min_rmsd = calc.oneVsTheOthers(1).min()
        
        
    saved.append(this_pose[:,:3])
    saved_phis.append(this_phis)
    saved_psis.append(this_psis)
        
    

In [85]:
# Dump the poses
folder = "/home/bcov/temp/tempmpp"
try:
    os.makedirs(folder)
except:
    pass

for i in range(len(saved_phis)):
    
    saved_phi = saved_phis[i]
    saved_psi = saved_psis[i]
    
    this_npose = npose.copy()
    this_phis = get_npose_phis(npose)
    this_psis = get_npose_psis(npose)
    this_tpose = tpose_from_npose(npose)
    
    for resnum in range(nsize(this_npose)):
        this_npose, this_tpose, this_phis = set_npose_phi(this_npose, this_tpose, this_phis, resnum, saved_phi[resnum])
        this_npose, this_tpose, this_psis = set_npose_psi(this_npose, this_tpose, this_psis, resnum, saved_psi[resnum])
    
    dump_npdb(this_npose, os.path.join(folder, "%i.pdb"%i))

In [86]:
# Speed tests
%timeit set_ca_phi(this_pose, this_tpose, this_phis, res, phi)

106 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [87]:
def to_time():
    calc = fast_rmsd_calc.RMSDCalculator("QCP_SERIAL_CALCULATOR", np.array([this_pose[:,:3]] + saved))
    min_rmsd = calc.oneVsTheOthers(1).min()
%timeit to_time()

724 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
